# Real-Time Violence Detection System


### Overview
This notebook tries to apply real-time violence detection system using pose estimation and deep learning. The system processes video streams to detect violent activities with visual outputs.

### Key Features
- **YOLOv8 Pose Estimation**: Multi-person pose detection
- **Biomechanical Feature Extraction**: 10-dimensional feature space
- **Real-time Processing**: Live webcam or video file input
- **Time series graph**: Time-series graphs and keyframe capture

### System Architecture
- Fixed 75-frame processing window
- Savitsky-Golay feature smoothing
- Multi-person tracking
- Violence probability scoring
- Multiple visualisation outputs

# Import Libraries

In [5]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from ultralytics import YOLO
from collections import deque, defaultdict
import pickle
from scipy.signal import savgol_filter
from scipy.spatial.distance import euclidean
import os
import time
import matplotlib.pyplot as plt

print("Libraries imported successfully")
print(f"TensorFlow version: {tf.__version__}")
print(f"OpenCV version: {cv2.__version__}")

Libraries imported successfully
TensorFlow version: 2.10.1
OpenCV version: 4.12.0


# Configuration

In [6]:
MODEL_PATH = "models/violence_detection_slr_aligned.keras"
SCALER_PATH = "extracted_features_slr_aligned/slr_aligned_scaler.pkl"

# Violence levels configuration
VIOLENCE_LEVELS = {
    0: ("Normal", (0, 0.20), (0, 255, 0)),
    1: ("Early Aggression", (0.21, 0.40), (0, 255, 255)),
    2: ("Pre-Violence", (0.41, 0.60), (0, 165, 255)),
    3: ("High Possibility", (0.61, 0.80), (0, 0, 255)),
    4: ("Active Violence", (0.81, 1.00), (0, 0, 139))
}

print("Configuration loaded")

Configuration loaded


# Core Detector Class

In [7]:
class ViolenceDetector:
    def __init__(self, model_path, scaler_path):
        # Load model and scaler
        self.model = load_model(model_path)
        with open(scaler_path, 'rb') as f:
            self.scaler = pickle.load(f)
        
        # Initialise YOLO
        self.pose_model = YOLO("yolov8l-pose.pt")
        
        # Training-aligned parameters
        self.CONFIDENCE_THRESHOLD = 0.25
        self.FEATURE_DIM = 10
        self.sequence_length = 75  # Fixed to match training
        
        # Tracking variables
        self.person_sequences = defaultdict(lambda: deque(maxlen=150))
        self.tracked_persons = {}
        self.next_person_id = 0
        
        # Violence tracking
        self.max_violence_prob = 0.0
        self.max_violence_level = "Normal"
        
        # Visualisation
        self.violence_history = []
        self.keyframe_captures = []
        
        print("Detector initialised successfully")

print("Detector class defined")

Detector class defined


# Feature Extraction

In [8]:
class ViolenceDetector:
    def __init__(self, model_path, scaler_path):
        # Load model and scaler
        self.model = load_model(model_path)
        with open(scaler_path, 'rb') as f:
            self.scaler = pickle.load(f)
        
        # Initialise YOLO
        self.pose_model = YOLO("yolov8l-pose.pt")
        
        # Training-aligned parameters
        self.CONFIDENCE_THRESHOLD = 0.25
        self.FEATURE_DIM = 10
        self.sequence_length = 75  # Fixed to match training
        
        # Tracking variables
        self.person_sequences = defaultdict(lambda: deque(maxlen=150))
        self.tracked_persons = {}
        self.next_person_id = 0
        
        # Violence tracking
        self.max_violence_prob = 0.0
        self.max_violence_level = "Normal"
        
        # Visualisation
        self.violence_history = []
        self.keyframe_captures = []
        
        print("Detector initialised successfully")

print("Detector class defined")

Detector class defined


# Processing and Prediction

In [11]:
class ViolenceDetector(ViolenceDetector):
    def process_frame(self, frame):
        """Process single frame for violence detection"""
        # YOLO pose detection
        results = self.pose_model(frame, conf=self.CONFIDENCE_THRESHOLD, verbose=False)
        
        if not results or not hasattr(results[0], 'keypoints'):
            return
        
        # Process detections and extract features
        # ... (feature extraction and tracking logic)
        
        # Make predictions when buffer is full
        if self.sequence_length and len(self.person_sequences[person_id]) >= self.sequence_length:
            probability = self.predict_violence(person_id)
            self.update_violence_levels(person_id, probability)
    
    def predict_violence(self, person_id):
        """Predict violence probability using trained model"""
        sequence = list(self.person_sequences[person_id])[-self.sequence_length:]
        sequence = np.array(sequence, dtype=np.float32)
        
        # Apply scaler and reshape
        sequence_flat = sequence.reshape(-1, self.FEATURE_DIM)
        sequence_scaled = self.scaler.transform(sequence_flat)
        sequence_reshaped = sequence_scaled.reshape(1, self.sequence_length, self.FEATURE_DIM)
        
        # Model prediction
        prediction = self.model.predict(sequence_reshaped, verbose=0)
        return float(prediction[0][0])

print("Processing methods added")

Processing methods added


# Visualisation

In [12]:
class ViolenceDetector(ViolenceDetector):
    def draw_visualisation(self, frame):
        """Draw detection results on frame"""
        h, w = frame.shape[:2]
        
        # Draw bounding boxes and skeletons
        for person_id, person_data in self.tracked_persons.items():
            bbox = person_data['bbox']
            violence_info = self.person_violence_levels.get(person_id, {"prob": 0.0, "level": "Normal", "color": (0, 255, 0)})
            
            # Draw bounding box
            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), violence_info["color"], 3)
            
            # Add label
            label = f"ID:{person_id} {violence_info['level']} ({violence_info['prob']:.3f})"
            cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
        
        # Add global info panel
        cv2.rectangle(frame, (10, 10), (400, 100), (0,0,0), -1)
        cv2.putText(frame, f"MAX: {self.max_violence_level} ({self.max_violence_prob:.3f})", 
                   (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        
        return frame

print("Visualisation methods added")

Visualisation methods added


# Execution

In [13]:
def run_detection():
    """Main function to run violence detection"""
    # Check files exist
    if not all(os.path.exists(p) for p in [MODEL_PATH, SCALER_PATH]):
        print("ERROR: Model or scaler not found. Please train model first.")
        return
    
    # Initialize detector
    detector = ViolenceDetector(MODEL_PATH, SCALER_PATH)
    
    # Input selection
    print("Select input source:")
    print("1. Webcam")
    print("2. Video file")
    choice = input("Enter choice (1 or 2): ").strip()
    
    if choice == "1":
        cap = cv2.VideoCapture(0)
    elif choice == "2":
        video_path = input("Enter video path: ").strip()
        cap = cv2.VideoCapture(video_path)
    else:
        print("Invalid choice")
        return
    
    print("Starting detection... Press 'q' to quit")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Process frame
        detector.process_frame(frame)
        
        # Draw results
        frame = detector.draw_visualization(frame)
        
        # Display
        cv2.imshow('Violence Detection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    print("Detection completed")

print("Main function defined")

Main function defined


# Run the system

In [ ]:
if __name__ == "__main__":
    run_detection()